### Required Libraries

In [9]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [10]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


### Drop Null target records

In [11]:
def map_weight(value):
    if (value >= '2022-10') and (value <= '2023-03'):
        return 2
    elif (value >= '2021-01') and (value <= '2021-06'):
        return 0.5
    else:
        return 1

model_data['Weight_vintage'] = model_data['month_year'].apply(map_weight)


In [12]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [13]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [14]:
df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    109935
1.0     13432
Name: count, dtype: int64

In [15]:
df['user_segment'].value_counts()

user_segment
OFFUS    90704
ONUS     32663
Name: count, dtype: int64

### Split the Time interval

In [16]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [17]:
model_df1.shape, oot_df1.shape

((101984, 2832), (17343, 2832))

In [18]:
features_list = [

#Bureau Intutive
    
'num_PERSONAL LOAN_enq_last_6M',
'num_CONSUMER LOAN_enq_last_6M_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'num_CREDIT CARD_enq_last_12M',
'Utilization_all',
'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'Utilization_CREDIT CARD_opened_last_36M',
'total_emi',
'total_credit_all_open',
    
# banking Intutive
'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',

#new banking features
'ratio_balance_after_2_days_max_credit_M3',
'ratio_5th_day_balance_to_max_credit_M3',
                 
#Loan application Features
'deviation_from_median_income/standard_deviation_income#last_365_Days',
'loan_amount_to_income#last_365_Days',
    
    
#ID Features
'aggs907',
'dm206s',
'g406s',

# New BUreau features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
           
'month_year',
    'Weight_vintage',
'60_in_15mob']

In [20]:
len(features_list)

34

In [21]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [22]:
oot_df = oot_df1[features_list]

In [23]:
model_df.shape, oot_df.shape

((101984, 34), (17343, 34))

In [24]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [25]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [26]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [27]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [28]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3163147/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [29]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3163147/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [30]:
X_oot.shape,y_oot.shape

((17343, 32), (17343,))

### Split Model dataset into Train and Test

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [32]:
X_train.shape, X_test.shape

((61190, 32), (40794, 32))

In [33]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 32) (12238, 32) (40794, 32)


In [34]:
# Get Vinateg weight values 
train_weights = X_train_new['Weight_vintage'].values
eval_weights = X_eval['Weight_vintage'].values
test_weights = X_test['Weight_vintage'].values
oot_weights = X_oot['Weight_vintage'].values

In [35]:
X_train_new = X_train_new.drop(columns=['Weight_vintage'])
X_eval = X_eval.drop(columns=['Weight_vintage'])
X_test = X_test.drop(columns=['Weight_vintage'])
X_oot = X_oot.drop(columns=['Weight_vintage'])

In [36]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new, weight=train_weights)
deval = xgb.DMatrix(X_eval, label=y_eval,weight=eval_weights)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [37]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,

    'lambda': 6.8,
    'alpha': 4.6,
    'gamma': 6.2,
   
     'scale_pos_weight':2.3,
    'random_state': 24
    
}



evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 800,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.61220	eval-auc:0.60573
[10]	train-auc:0.66343	eval-auc:0.65634
[20]	train-auc:0.66408	eval-auc:0.65646
[30]	train-auc:0.66405	eval-auc:0.65687
[40]	train-auc:0.66450	eval-auc:0.65639
[50]	train-auc:0.66491	eval-auc:0.65625
[60]	train-auc:0.66740	eval-auc:0.65830
[70]	train-auc:0.66763	eval-auc:0.65875
[80]	train-auc:0.66841	eval-auc:0.65959
[90]	train-auc:0.66995	eval-auc:0.66114
[100]	train-auc:0.67117	eval-auc:0.66153
[110]	train-auc:0.67267	eval-auc:0.66262
[120]	train-auc:0.67381	eval-auc:0.66374
[130]	train-auc:0.67477	eval-auc:0.66453
[140]	train-auc:0.67566	eval-auc:0.66530
[150]	train-auc:0.67642	eval-auc:0.66564
[160]	train-auc:0.67716	eval-auc:0.66633
[170]	train-auc:0.67810	eval-auc:0.66715
[180]	train-auc:0.67908	eval-auc:0.66823
[190]	train-auc:0.67987	eval-auc:0.66872
[200]	train-auc:0.68071	eval-auc:0.66944
[210]	train-auc:0.68150	eval-auc:0.67013
[220]	train-auc:0.68208	eval-auc:0.67077
[230]	train-auc:0.68276	eval-auc:0.67123
[240]	train-auc:0.68342	eva

In [38]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.70, Gini: 0.40, KS: 0.29
Eval AUC: 0.68, Gini: 0.36, KS: 0.27
Test AUC: 0.68, Gini: 0.36, KS: 0.26
OOT AUC: 0.70, Gini: 0.40, KS: 0.30


In [39]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.70, Gini: 0.4010, KS: 0.2906
Test AUC: 0.68, Gini: 0.3641, KS: 0.2631
OOT AUC: 0.70, Gini: 0.4048, KS: 0.2989


In [32]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
2              Num_of_PERSONAL LOAN_opened_last6M_12M       195.0
23                                              g406s       183.0
28                           No of enq in past 90days       150.0
6                highest_sanction_amount_credit_cards       137.0
1                   num_CONSUMER LOAN_enq_last_6M_36M       117.0
21                                            aggs907       115.0
29                          Num_Enq_non_BANK_last_12M       111.0
19  deviation_from_median_income/standard_deviatio...       110.0
24                         percentage_ever_delinquent       105.0
7                          Age_of_oldest_open_account       100.0
11                         bounced_transactions_count        95.0
4                                     Utilization_all        94.0
5   Total_outstanding_Balance_CREDIT CARD_opened_l...        85.0
8             Utilization_CREDIT CARD_opened_last_36M        77.0
9         

In [33]:
feature_importance_df

Feature  Importance
2              Num_of_PERSONAL LOAN_opened_last6M_12M       195.0
23                                              g406s       183.0
28                           No of enq in past 90days       150.0
6                highest_sanction_amount_credit_cards       137.0
1                   num_CONSUMER LOAN_enq_last_6M_36M       117.0
21                                            aggs907       115.0
29                          Num_Enq_non_BANK_last_12M       111.0
19  deviation_from_median_income/standard_deviatio...       110.0
24                         percentage_ever_delinquent       105.0
7                          Age_of_oldest_open_account       100.0
11                         bounced_transactions_count        95.0
4                                     Utilization_all        94.0
5   Total_outstanding_Balance_CREDIT CARD_opened_l...        85.0
8             Utilization_CREDIT CARD_opened_last_36M        77.0
9                                           total_emi        69.0
12                           Min_Balance_Min_M1_M2_M3        69.0
22                                             dm206s        65.0
3                        num_CREDIT CARD_enq_last_12M        64.0
30                     Num_of_installment_loan_opened        56.0
20                loan_amount_to_income#last_365_Days        51.0
10                              total_credit_all_open        47.0
27                       percentage_utilization_gt_75        44.0
13                               num_credit_gt_salary        42.0
26           months_since_most_recent_delinquency_all        40.0
25   months_since_most_recent_delinquency_installment        40.0
18             ratio_5th_day_balance_to_max_credit_M3        39.0
17           ratio_balance_after_2_days_max_credit_M3        27.0
15                            Max_Credit_Avg_M1_M2_M3        20.0
14                                    upi_trans_count        15.0
16               ratio_total_debit_to_total_credit_M3        15.0
0                       num_PERSONAL LOAN_enq_last_6M        12.0

In [34]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.284722         0.434954        0.737514        0.347555   4896   
9          0.176915         0.308728        0.347483        0.277126   4895   
8          0.138509         0.254000        0.277119        0.233839   4895   
7          0.114402         0.218299        0.233831        0.203967   4895   
6          0.089275         0.191736        0.203966        0.180220   4895   
5          0.080899         0.170495        0.180212        0.161313   4895   
4          0.070684         0.153072        0.161310        0.145026   4895   
3          0.053524         0.137907        0.145026        0.130576   4895   
2          0.041675         0.122058        0.130569        0.112085   4895   
1          0.028391         0.097483        0.112074        0.060584   4896   

        label_good  label_bad  
Decile                         
10            3502       1394  
9             4029        866  
8             4217        678  
7             4335        560  
6             4458        437  
5             4499        396  
4             4549        346  
3             4633        262  
2             4691        204  
1             4757        139

In [35]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.267892         0.433825        0.734723        0.346954   4080   
9          0.172836         0.307998        0.346953        0.276303   4079   
8          0.141701         0.254218        0.276302        0.233897   4079   
7          0.120343         0.217999        0.233871        0.203535   4080   
6          0.099534         0.191455        0.203505        0.180408   4079   
5          0.083354         0.170834        0.180406        0.161795   4079   
4          0.078922         0.153558        0.161791        0.145686   4080   
3          0.060554         0.138149        0.145679        0.130630   4079   
2          0.047561         0.122157        0.130626        0.112437   4079   
1          0.033578         0.097840        0.112437        0.065083   4080   

        label_good  label_bad  
Decile                         
10            2987       1093  
9             3374        705  
8             3501        578  
7             3589        491  
6             3673        406  
5             3739        340  
4             3758        322  
3             3832        247  
2             3885        194  
1             3943        137

In [40]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.271470         0.439735        0.733754        0.353056   1735   
9          0.177624         0.312605        0.353053        0.279646   1734   
8          0.135525         0.255783        0.279637        0.234656   1734   
7          0.118800         0.217436        0.234634        0.202147   1734   
6          0.092849         0.188432        0.202132        0.176268   1734   
5          0.082997         0.165701        0.176259        0.156177   1735   
4          0.059400         0.147874        0.156174        0.139960   1734   
3          0.054210         0.132680        0.139950        0.125457   1734   
2          0.038639         0.117129        0.125443        0.107627   1734   
1          0.028242         0.092555        0.107586        0.056902   1735   

        label_good  label_bad  
Decile                         
10            1264        471  
9             1426        308  
8             1499        235  
7             1528        206  
6             1573        161  
5             1591        144  
4             1631        103  
3             1640         94  
2             1667         67  
1             1686         49

In [41]:
#Save the Model
xgb_model.save_model('LeoPayu_BBILA_60_in_15mob_Vin.json')
print('Model Saved')

Model Saved


In [42]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBILA_Train_Pred_Vin.csv',index=False)
Eval.to_csv('LeoPayu_BBILA_Eval_Pred_Vin.csv',index=False)
Test.to_csv('LeoPayu_BBILA_Test_Pred_Vin.csv',index=False)
Oot.to_csv('LeoPayu_BBILA_Oot_Pred_Vin.csv',index=False)
